### Import library

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession


### Initialize spark session

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.task.maxDirectResultSize", "4m") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/01/25 14:10:29 WARN Utils: Your hostname, Device02072004 resolves to a loopback address: 127.0.1.1; using 172.24.50.213 instead (on interface eth0)
25/01/25 14:10:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/25 14:10:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/25 14:10:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/25 14:10:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Read dataset

In [ ]:
df_train = spark.read.csv("Cleaned_dataset.csv", header=True, inferSchema=True)
print(df_train.count())

25/01/25 14:11:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df_train.show(20)

+---------------+-----------+---------+-----------+-------+------+------------+-----------+------------+-----------+-----------------+---------+----+
|Date_of_journey|Journey_day|  Airline|Flight_code|  Class|Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|Days_left|Fare|
+---------------+-----------+---------+-----------+-------+------+------------+-----------+------------+-----------+-----------------+---------+----+
|     2023-01-16|     Monday| SpiceJet|    SG-8169|Economy| Delhi|  After 6 PM|   non-stop|  After 6 PM|     Mumbai|           2.0833|        1|5335|
|     2023-01-16|     Monday|   Indigo|    6E-2519|Economy| Delhi|  After 6 PM|   non-stop| Before 6 AM|     Mumbai|           2.3333|        1|5899|
|     2023-01-16|     Monday| GO FIRST|     G8-354|Economy| Delhi|  After 6 PM|   non-stop| Before 6 AM|     Mumbai|           2.1667|        1|5801|
|     2023-01-16|     Monday| SpiceJet|    SG-8709|Economy| Delhi|  After 6 PM|   non-stop|  After 6

### Preprocess

In [1]:
df_train = df_train.dropDuplicates()
print(df_train.count())

NameError: name 'df_train' is not defined

In [6]:
df_train.describe().show()

25/01/25 13:03:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------+---------+--------------------+---------------+---------+------------+-----------+------------+-----------+-----------------+------------------+------------------+
|summary|Journey_day|  Airline|         Flight_code|          Class|   Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|         Days_left|              Fare|
+-------+-----------+---------+--------------------+---------------+---------+------------+-----------+------------+-----------+-----------------+------------------+------------------+
|  count|     445366|   445366|              445366|         445366|   445366|      445366|     445366|      445366|     445366|           445366|            445366|            445366|
|   mean|       NULL|     NULL|6.478977189753123...|           NULL|     NULL|        NULL|       NULL|        NULL|       NULL|12.18249122407184|25.617822195677263|22919.747165252847|
| stddev|       NULL|     NULL|4.84548855775472E...|           NULL|     NU

In [7]:
from pyspark.sql.functions import col, sum

# Đếm số lượng giá trị null trong mỗi cột
null_counts = df_train.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in df_train.columns]
)

# Hiển thị kết quả
null_counts.show()


+---------------+-----------+-------+-----------+-----+------+---------+-----------+-------+-----------+-----------------+---------+----+
|Date_of_journey|Journey_day|Airline|Flight_code|Class|Source|Departure|Total_stops|Arrival|Destination|Duration_in_hours|Days_left|Fare|
+---------------+-----------+-------+-----------+-----+------+---------+-----------+-------+-----------+-----------------+---------+----+
|              0|          0|      0|          0|    0|     0|        0|          0|      0|          0|                0|        0|   0|
+---------------+-----------+-------+-----------+-----+------+---------+-----------+-------+-----------+-----------------+---------+----+



In [8]:
from pyspark.sql.functions import col

# Đếm số lần xuất hiện của từng giá trị trong cột Total_stops
df_train.groupBy("Fare").count().orderBy(col("count").desc()).show()


+-----+-----+
| Fare|count|
+-----+-----+
|54879| 2702|
|54608| 2521|
|49613| 2128|
|49713| 2089|
|41549| 1842|
|42457| 1772|
|14447| 1743|
|50289| 1708|
|42716| 1604|
|49725| 1536|
| 8895| 1360|
|60703| 1323|
|45345| 1271|
| 5955| 1240|
|16494| 1159|
|45257| 1158|
|12262| 1140|
|60996| 1134|
|46118| 1086|
| 9840| 1069|
+-----+-----+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import split, col

# Tách cột 'Date_of_journey' và tạo cột mới 'Journey_month'
df_train = df_train.withColumn("Journey_month", split(col("Date_of_journey"), "-")[1].cast("int"))

# Loại bỏ cột 'Date_of_journey'
df_train = df_train.drop("Date_of_journey")

# Hiển thị kết quả
df_train.show(20)


+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+
|Journey_day|  Airline|Flight_code|          Class|Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|Days_left| Fare|Journey_month|
+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+
|     Monday|   Indigo|    6E-2328|        Economy| Delhi|6 AM - 12 PM|   non-stop|12 PM - 6 PM|     Mumbai|           2.5833|        1| 8894|            1|
|     Monday|Air India|     AI-544|        Economy| Delhi|12 PM - 6 PM|     1-stop| Before 6 AM|     Mumbai|             7.25|        1|11205|            1|
|     Monday|  Vistara|     UK-809|       Business| Delhi|  After 6 PM|     1-stop|6 AM - 12 PM|     Mumbai|          12.4167|        1|73414|            1|
|  Wednesday|Air India|     AI-883|        Economy| Delhi|

In [ ]:
from pyspark.sql import functions as f

# Add a new column 'Weekend' based on the 'Journey_day' column
df_train = df_train.withColumn("Weekend", f.when(df_train["Journey_day"] == "Sunday", 1).otherwise(0))



+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|Journey_day|  Airline|Flight_code|          Class|Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|Days_left| Fare|Journey_month|Weekend|
+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|     Monday|   Indigo|    6E-2328|        Economy| Delhi|6 AM - 12 PM|   non-stop|12 PM - 6 PM|     Mumbai|           2.5833|        1| 8894|            1|      0|
|     Monday|Air India|     AI-544|        Economy| Delhi|12 PM - 6 PM|     1-stop| Before 6 AM|     Mumbai|             7.25|        1|11205|            1|      0|
|     Monday|  Vistara|     UK-809|       Business| Delhi|  After 6 PM|     1-stop|6 AM - 12 PM|     Mumbai|          12.4167|        1|73414|            1|      0|
|  Wednesd

In [11]:
df_train.show(20)

+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|Journey_day|  Airline|Flight_code|          Class|Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|Days_left| Fare|Journey_month|Weekend|
+-----------+---------+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|     Monday|   Indigo|    6E-2328|        Economy| Delhi|6 AM - 12 PM|   non-stop|12 PM - 6 PM|     Mumbai|           2.5833|        1| 8894|            1|      0|
|     Monday|Air India|     AI-544|        Economy| Delhi|12 PM - 6 PM|     1-stop| Before 6 AM|     Mumbai|             7.25|        1|11205|            1|      0|
|     Monday|  Vistara|     UK-809|       Business| Delhi|  After 6 PM|     1-stop|6 AM - 12 PM|     Mumbai|          12.4167|        1|73414|            1|      0|
|  Wednesd

In [12]:
# Drop the 'Airline' and 'Flight_code' columns
df_train = df_train.drop("Airline", "Flight_code")

# Show the result
df_train.show(20)


+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|Journey_day|          Class|Source|   Departure|Total_stops|     Arrival|Destination|Duration_in_hours|Days_left| Fare|Journey_month|Weekend|
+-----------+---------------+------+------------+-----------+------------+-----------+-----------------+---------+-----+-------------+-------+
|     Monday|        Economy| Delhi|6 AM - 12 PM|   non-stop|12 PM - 6 PM|     Mumbai|           2.5833|        1| 8894|            1|      0|
|     Monday|        Economy| Delhi|12 PM - 6 PM|     1-stop| Before 6 AM|     Mumbai|             7.25|        1|11205|            1|      0|
|     Monday|       Business| Delhi|  After 6 PM|     1-stop|6 AM - 12 PM|     Mumbai|          12.4167|        1|73414|            1|      0|
|  Wednesday|        Economy| Delhi|  After 6 PM|     1-stop|12 PM - 6 PM|     Mumbai|          18.8333|        3|11344|            1|      0|

In [13]:
df_train.dtypes

[('Journey_day', 'string'),
 ('Class', 'string'),
 ('Source', 'string'),
 ('Departure', 'string'),
 ('Total_stops', 'string'),
 ('Arrival', 'string'),
 ('Destination', 'string'),
 ('Duration_in_hours', 'double'),
 ('Days_left', 'int'),
 ('Fare', 'int'),
 ('Journey_month', 'int'),
 ('Weekend', 'int')]

### Change dtype

In [14]:
# Get the list of columns and their data types
columns_and_types = df_train.dtypes
# Separate categorical and numerical columns
categorical_columns = [col for col, dtype in columns_and_types if dtype == 'string']
numerical_columns = [col for col, dtype in columns_and_types if dtype in ['int', 'double']]

# Select categorical data
train_categorical_data_pyspark = df_train.select(*categorical_columns)

# Select numerical data
train_numerical_data_pyspark = df_train.select(*numerical_columns)

In [15]:
train_numerical_data_pyspark.show(20) 

+-----------------+---------+-----+-------------+-------+
|Duration_in_hours|Days_left| Fare|Journey_month|Weekend|
+-----------------+---------+-----+-------------+-------+
|           2.5833|        1| 8894|            1|      0|
|             7.25|        1|11205|            1|      0|
|          12.4167|        1|73414|            1|      0|
|          18.8333|        3|11344|            1|      0|
|           2.1667|        5| 5801|            1|      0|
|           5.0833|        5| 7330|            1|      0|
|          17.1667|        5|31462|            1|      0|
|          17.4167|        6|41113|            1|      0|
|          11.5833|        7|21200|            1|      1|
|             26.5|        7|55696|            1|      1|
|             11.5|        8| 5291|            1|      0|
|             7.25|        8|41113|            1|      0|
|          14.6667|        8|41281|            1|      0|
|           2.5833|       10| 6682|            1|      0|
|             

In [11]:
# train_categorical_data_pyspark = spark.createDataFrame(train_categorical_data)
# train_numerical_data_pyspark = spark.createDataFrame(train_numerical_data)
# df_pyspark = spark.createDataFrame(df_train)

In [16]:
train_numerical_data_pyspark.printSchema()

root
 |-- Duration_in_hours: double (nullable = true)
 |-- Days_left: integer (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Journey_month: integer (nullable = true)
 |-- Weekend: integer (nullable = false)



In [17]:
train_categorical_data_pyspark.printSchema()

root
 |-- Journey_day: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Departure: string (nullable = true)
 |-- Total_stops: string (nullable = true)
 |-- Arrival: string (nullable = true)
 |-- Destination: string (nullable = true)



In [18]:
df_train.printSchema()

root
 |-- Journey_day: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Departure: string (nullable = true)
 |-- Total_stops: string (nullable = true)
 |-- Arrival: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Duration_in_hours: double (nullable = true)
 |-- Days_left: integer (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Journey_month: integer (nullable = true)
 |-- Weekend: integer (nullable = false)



In [19]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_iindex",handleInvalid="keep") for col in train_categorical_data_pyspark.columns]
encoders = [OneHotEncoder(inputCol=f"{col}_iindex", outputCol=f"{col}Vec",handleInvalid="keep") for col in train_categorical_data_pyspark.columns]

In [20]:
feature_columns = [col + "Vec" for col in train_categorical_data_pyspark.columns] + [col for col in train_numerical_data_pyspark.columns] 
feature_columns.remove("Fare")
# Assemble features
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [21]:
# feature_columns.pop(9)
feature_columns

['Journey_dayVec',
 'ClassVec',
 'SourceVec',
 'DepartureVec',
 'Total_stopsVec',
 'ArrivalVec',
 'DestinationVec',
 'Duration_in_hours',
 'Days_left',
 'Journey_month',
 'Weekend']

### Linear Regression

In [22]:
# Define Logistic Regression model
from pyspark.ml.regression import LinearRegression 
lr = LinearRegression(featuresCol="features", labelCol="Fare",maxIter=30, regParam=0.3, elasticNetParam=0.8)
# Create pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])


In [23]:
df_pyspark = df_train.repartition(200)
train_set, test_set = df_pyspark.randomSplit([0.8, 0.2], seed=42)

In [25]:
model = pipeline.fit(train_set)

25/01/25 13:20:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/25 13:20:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


### Evaluate Linear Regression

In [44]:
train_pred = model.transform(test_set)

In [45]:
train_pred.select("features", "Fare", "prediction").show()

+--------------------+------+------------------+
|            features|  Fare|        prediction|
+--------------------+------+------------------+
|(47,[2,9,19,23,26...| 75780|55111.660733053795|
|(47,[2,9,19,23,26...| 52215|54438.682036834085|
|(47,[2,9,19,23,26...| 35587|51342.043518405335|
|(47,[2,9,19,23,26...| 33828| 45834.35634207111|
|(47,[2,9,19,23,26...| 41508| 53545.88953672675|
|(47,[2,9,19,23,26...| 41508| 52165.33005680445|
|(47,[2,9,19,23,26...| 53204| 47703.16518286342|
|(47,[2,9,19,23,26...| 68052| 56423.37585885548|
|(47,[2,9,19,21,26...| 68724| 52269.31857352003|
|(47,[2,9,19,21,26...| 59652| 54503.09771967231|
|(47,[2,9,19,21,26...| 61302| 50162.28251023708|
|(47,[2,9,19,21,26...| 47220| 52504.81778319011|
|(47,[2,9,19,21,26...| 41780| 50367.41838733596|
|(47,[2,9,19,21,26...| 47332|  58207.9039480182|
|(47,[2,9,19,21,26...| 41508|51394.943585044995|
|(47,[2,9,19,21,26...| 29292|50126.718058910665|
|(47,[2,9,19,21,26...| 52215|54159.488167386386|
|(47,[2,9,19,21,26..

In [46]:
from pyspark.ml.evaluation import RegressionEvaluator

# Khởi tạo evaluator cho các chỉ số
evaluator_rmse = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="r2")

# Tính toán các chỉ số
rmse = evaluator_rmse.evaluate(train_pred)
mae = evaluator_mae.evaluate(train_pred)
r2 = evaluator_r2.evaluate(train_pred)

# In kết quả
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")


Root Mean Squared Error (RMSE): 8027.679286892896
Mean Absolute Error (MAE): 5254.926574079209
R-squared (R²): 0.8457770393492066


### Random forest

In [28]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="features", labelCol="Fare", maxBins=32, maxDepth=8,minInstancesPerNode=1, \
                           minInfoGain=0.01, maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, featureSubsetStrategy='auto', \
                           seed=42, subsamplingRate=1.0)   
pipeline3 = Pipeline(stages=indexers + encoders + [assembler, rf])

In [29]:
train_set, test_set = df_train.randomSplit([0.8, 0.2], seed=42)
model_rf = pipeline3.fit(train_set)

25/01/25 13:25:27 WARN DAGScheduler: Broadcasting large task binary with size 1112.5 KiB


### Evaluate Random forest

In [42]:
train_pred = model_rf.transform(test_set)
train_pred.select("features", "Fare", "prediction").show()

+--------------------+------+------------------+
|            features|  Fare|        prediction|
+--------------------+------+------------------+
|(47,[1,9,19,23,26...| 75780| 53109.35826214447|
|(47,[1,9,19,23,26...| 52215| 57822.68091134554|
|(47,[1,9,19,23,26...| 35587|  51294.7894038727|
|(47,[1,9,19,23,26...| 33828| 45591.26629063041|
|(47,[1,9,19,23,26...| 41508| 54885.78492264815|
|(47,[1,9,19,23,26...| 41508| 49518.62818442413|
|(47,[1,9,19,23,26...| 53204| 46085.93492191007|
|(47,[1,9,19,23,26...| 68052| 60901.20298859492|
|(47,[1,9,19,21,26...| 68724|52198.744833209435|
|(47,[1,9,19,21,26...| 59652|53114.067421182335|
|(47,[1,9,19,21,26...| 61302| 52031.48399213698|
|(47,[1,9,19,21,26...| 47220| 52031.48399213698|
|(47,[1,9,19,21,26...| 41780| 47363.54378333789|
|(47,[1,9,19,21,26...| 47332|60170.288903673376|
|(47,[1,9,19,21,26...| 41508| 52229.38479709196|
|(47,[1,9,19,21,26...| 29292| 49030.00113806026|
|(47,[1,9,19,21,26...| 52215| 52771.15867159238|
|(47,[1,9,19,21,26..

In [43]:
from pyspark.ml.evaluation import RegressionEvaluator

# Khởi tạo evaluator cho các chỉ số
evaluator_rmse = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="r2")

# Tính toán các chỉ số
rmse = evaluator_rmse.evaluate(train_pred)
mae = evaluator_mae.evaluate(train_pred)
r2 = evaluator_r2.evaluate(train_pred)

# In kết quả
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")
    

Root Mean Squared Error (RMSE): 7057.93126160127
Mean Absolute Error (MAE): 4355.3236787189
R-squared (R²): 0.8807869340356274


### GB

In [32]:
from pyspark.ml.regression import GBTRegressor

gb = GBTRegressor(featuresCol="features", labelCol="Fare", maxBins=32, maxIter=30 ,minInstancesPerNode=1, \
                  minInfoGain=0.01, maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, stepSize=0.1, \
                  seed=42,subsamplingRate=1.0, impurity='variance', featureSubsetStrategy='all', validationTol=0.01)
pipeline4 = Pipeline(stages=indexers + encoders  + [assembler, gb])

In [33]:
train_set, test_set = df_train.randomSplit([0.8, 0.2], seed=42)
model_gb = pipeline4.fit(train_set)

### Evaluate GB

In [40]:
train_pred = model_gb.transform(test_set)
train_pred.select("features", "Fare", "prediction").show()

+--------------------+------+------------------+
|            features|  Fare|        prediction|
+--------------------+------+------------------+
|(47,[1,9,19,23,26...| 75780|  50371.1836527247|
|(47,[1,9,19,23,26...| 52215|56116.142918251484|
|(47,[1,9,19,23,26...| 35587| 49994.66737092154|
|(47,[1,9,19,23,26...| 33828|39576.865414777494|
|(47,[1,9,19,23,26...| 41508|54779.322204949596|
|(47,[1,9,19,23,26...| 41508| 43773.93282664018|
|(47,[1,9,19,23,26...| 53204|45427.832032691746|
|(47,[1,9,19,23,26...| 68052| 61054.60547727537|
|(47,[1,9,19,21,26...| 68724| 52335.70592376635|
|(47,[1,9,19,21,26...| 59652| 53836.20359778763|
|(47,[1,9,19,21,26...| 61302|51409.300854806614|
|(47,[1,9,19,21,26...| 47220| 52536.88970523532|
|(47,[1,9,19,21,26...| 41780| 49407.88729843527|
|(47,[1,9,19,21,26...| 47332| 63678.99858590961|
|(47,[1,9,19,21,26...| 41508| 52739.71264689648|
|(47,[1,9,19,21,26...| 29292|41925.318152850145|
|(47,[1,9,19,21,26...| 52215|52245.158675389044|
|(47,[1,9,19,21,26..

In [41]:
from pyspark.ml.evaluation import RegressionEvaluator

# Khởi tạo evaluator cho các chỉ số
evaluator_rmse = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="r2")

# Tính toán các chỉ số
rmse = evaluator_rmse.evaluate(train_pred)
mae = evaluator_mae.evaluate(train_pred)
r2 = evaluator_r2.evaluate(train_pred)

# In kết quả
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

Root Mean Squared Error (RMSE): 6720.927969301248
Mean Absolute Error (MAE): 4145.848172692141
R-squared (R²): 0.8918995528048023


### Decision Tree

In [36]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol="features", labelCol="Fare", maxDepth=8, maxBins=32,minInstancesPerNode=1, minInfoGain=0.01, \
                           maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity='variance', seed=42)
pipeline5 = Pipeline(stages=indexers  + encoders +  [assembler, dt])

In [37]:
train_set, test_set = df_train.randomSplit([0.8, 0.2], seed=42)
model_dt = pipeline5.fit(train_set)

### Evaluate Decision Tree

In [38]:
train_pred = model_dt.transform(test_set)
train_pred.select("features", "Fare", "prediction").show()

+--------------------+------+------------------+
|            features|  Fare|        prediction|
+--------------------+------+------------------+
|(47,[1,9,19,23,26...| 75780| 44085.44668008048|
|(47,[1,9,19,23,26...| 52215| 60415.95928080381|
|(47,[1,9,19,23,26...| 35587| 51991.17093363479|
|(47,[1,9,19,23,26...| 33828|  40659.5974130962|
|(47,[1,9,19,23,26...| 41508|56228.718600228225|
|(47,[1,9,19,23,26...| 41508| 51991.17093363479|
|(47,[1,9,19,23,26...| 53204|  40659.5974130962|
|(47,[1,9,19,23,26...| 68052| 60415.95928080381|
|(47,[1,9,19,21,26...| 68724| 51991.17093363479|
|(47,[1,9,19,21,26...| 59652| 51991.17093363479|
|(47,[1,9,19,21,26...| 61302| 51991.17093363479|
|(47,[1,9,19,21,26...| 47220| 51991.17093363479|
|(47,[1,9,19,21,26...| 41780|  40659.5974130962|
|(47,[1,9,19,21,26...| 47332| 62625.23382045929|
|(47,[1,9,19,21,26...| 41508| 51991.17093363479|
|(47,[1,9,19,21,26...| 29292| 51991.17093363479|
|(47,[1,9,19,21,26...| 52215| 51991.17093363479|
|(47,[1,9,19,21,26..

In [39]:
from pyspark.ml.evaluation import RegressionEvaluator

# Khởi tạo evaluator cho các chỉ số
evaluator_rmse = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="Fare", predictionCol="prediction", metricName="r2")

# Tính toán các chỉ số
rmse = evaluator_rmse.evaluate(train_pred)
mae = evaluator_mae.evaluate(train_pred)
r2 = evaluator_r2.evaluate(train_pred)

# In kết quả
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

Root Mean Squared Error (RMSE): 7141.9533848904985
Mean Absolute Error (MAE): 4436.513036607309
R-squared (R²): 0.8779316620958744


In [47]:
model_path = 'model2'
model_gb.write().overwrite().save(model_path)